<a href="https://colab.research.google.com/github/Gaussiandra/NTA_Landmarks_Detection/blob/master/solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu May 28 15:53:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Импорт библиотек

In [1]:
!pip install pretrainedmodels

     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp36-none-any.whl size=60962 sha256=75ea6fcafb0a85c05b45d46379a1f6e93a7fbc4bc69a16387b278b6587e985aa
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
Successfully built pretrainedmodels


In [0]:
#import pretrainedmodels
import os

import torch

import pandas as pd
import numpy as np
import cv2
import gc
import time
import random

from torch.utils import data
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import tqdm
from torch.nn import functional as fnn
from torch.utils import data
from torchvision import transforms

import pickle

In [0]:
def restore_landmarks(landmarks, f, margins):
    dx, dy = margins
    landmarks[:, 0] += dx
    landmarks[:, 1] += dy
    landmarks /= f
    return landmarks


def restore_landmarks_batch(landmarks, fs, margins_x, margins_y):
    landmarks[:, :, 0] += margins_x[:, None]
    landmarks[:, :, 1] += margins_y[:, None]
    landmarks /= fs[:, None, None]
    return landmarks

## Параметры

In [0]:
TRAIN_SIZE = 0.8
NUM_PTS = 194
CROP_SIZE = 220
BATCH_SIZE = 64

In [0]:
SEED = 3462

random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [0]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
from google.colab import drive
drive.mount('/content/drive')

global_path = '/content/drive/My Drive/ML/NTA Landmarks Detection/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
TEST_PATH = global_path+"data/baseline_NTA/data/test/" 
TRAIN_PATH = global_path+"data/baseline_NTA/data/train/"
SUBMISSION_PATH = global_path+"data/baseline_NTA/data/train/"
LANDMARKS = global_path+"data/baseline_NTA/data/train/landmarks.csv"

In [0]:
SUBMISSION_HEADER = "file_name,Point_M0_X,Point_M0_Y,Point_M1_X,Point_M1_Y,Point_M2_X,Point_M2_Y,Point_M3_X,Point_M3_Y,Point_M4_X,Point_M4_Y,Point_M5_X,Point_M5_Y,Point_M6_X,Point_M6_Y,Point_M7_X,Point_M7_Y,Point_M8_X,Point_M8_Y,Point_M9_X,Point_M9_Y,Point_M10_X,Point_M10_Y,Point_M11_X,Point_M11_Y,Point_M12_X,Point_M12_Y,Point_M13_X,Point_M13_Y,Point_M14_X,Point_M14_Y,Point_M15_X,Point_M15_Y,Point_M16_X,Point_M16_Y,Point_M17_X,Point_M17_Y,Point_M18_X,Point_M18_Y,Point_M19_X,Point_M19_Y,Point_M20_X,Point_M20_Y,Point_M21_X,Point_M21_Y,Point_M22_X,Point_M22_Y,Point_M23_X,Point_M23_Y,Point_M24_X,Point_M24_Y,Point_M25_X,Point_M25_Y,Point_M26_X,Point_M26_Y,Point_M27_X,Point_M27_Y,Point_M28_X,Point_M28_Y,Point_M29_X,Point_M29_Y\n"

In [11]:
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.max_memory_allocated(device='cuda'))
print(torch.cuda.empty_cache())

1.5.0+cu101
10.1
7603
True
1
0
None


In [0]:
class ScaleMinSideToSize(object):
    def __init__(self, size=(CROP_SIZE, CROP_SIZE), elem_name='image'):
        self.size = torch.tensor(size, dtype=torch.float)
        self.elem_name = elem_name

    def __call__(self, sample):
        h, w, _ = sample[self.elem_name].shape
        if h > w:
            f = self.size[0] / w
        else:
            f = self.size[1] / h

        sample[self.elem_name] = cv2.resize(sample[self.elem_name], None, fx=f, fy=f, interpolation=cv2.INTER_AREA)
        sample["scale_coef"] = f

        if 'landmarks' in sample:
            landmarks = sample['landmarks'].reshape(-1, 2).float()
            landmarks = landmarks * f
            sample['landmarks'] = landmarks.reshape(-1)

        return sample


class CropCenter(object):
    def __init__(self, size=128, elem_name='image'):
        self.size = size
        self.elem_name = elem_name

    def __call__(self, sample):
        img = sample[self.elem_name]
        h, w, _ = img.shape
        margin_h = (h - self.size) // 2
        margin_w = (w - self.size) // 2
        sample[self.elem_name] = img[margin_h:margin_h + self.size, margin_w:margin_w + self.size]
        sample["crop_margin_x"] = margin_w
        sample["crop_margin_y"] = margin_h

        if 'landmarks' in sample:
            landmarks = sample['landmarks'].reshape(-1, 2)
            landmarks -= torch.tensor((margin_w, margin_h), dtype=landmarks.dtype)[None, :]
            sample['landmarks'] = landmarks.reshape(-1)

        return sample


class TransformByKeys(object):
    def __init__(self, transform, names):
        self.transform = transform
        self.names = set(names)

    def __call__(self, sample):
        for name in self.names:
            if name in sample:
                sample[name] = self.transform(sample[name])

        return sample


In [0]:
class LandmarksDataset(data.Dataset):
    def __init__(self, root, transforms, split="train"):
        super(LandmarksDataset, self).__init__()
        self.root = root
        landmark_file_name = os.path.join(root, 'landmarks.csv') if split is not "test" \
            else os.path.join(root, "test_points.csv")
        images_root = os.path.join(root, "images")

        self.image_names = []
        self.landmarks = []

        with open(landmark_file_name, "rt") as fp:
            num_lines = sum(1 for line in fp)
        num_lines -= 1  # header

        with open(landmark_file_name, "rt") as fp:
            for i, line in tqdm.tqdm(enumerate(fp)):
                if i == 0:
                    continue  # skip header
                if split == "train" and i == int(TRAIN_SIZE * num_lines):
                    break  # reached end of train part of data
                elif split == "val" and i < int(TRAIN_SIZE * num_lines):
                    continue  # has not reached start of val part of data
                elements = line.strip().split(",")
                image_name = os.path.join(images_root, elements[0])
                self.image_names.append(image_name)

                if split in ("train", "val"):
                    landmarks = list(map(np.int16, elements[1:]))
                    landmarks = np.array(landmarks, dtype=np.int16).reshape((len(landmarks) // 2, 2))
                    self.landmarks.append(landmarks)

        if split in ("train", "val"):
            self.landmarks = torch.as_tensor(self.landmarks)
        else:
            self.landmarks = None

        self.transforms = transforms

    def __getitem__(self, idx):
        sample = {}
        if self.landmarks is not None:
            landmarks = self.landmarks[idx]
            sample["landmarks"] = landmarks

        image = cv2.imread(self.image_names[idx])
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        sample["image"] = image

        if self.transforms is not None:
            sample = self.transforms(sample)

        return sample

    def __len__(self):
        return len(self.image_names)

In [0]:
def train(model, loader, loss_fn, optimizer,  device):
    model.train()
    train_loss = []
    
        
    for batch in tqdm.tqdm(loader, total=len(loader), desc="training..."):
        images = batch["image"].to(device)  # B x 3 x CROP_SIZE x CROP_SIZE
        landmarks = batch["landmarks"]  # B x (2 * NUM_PTS)

        pred_landmarks = model(images).cpu()  # B x (2 * NUM_PTS)
        loss = loss_fn(pred_landmarks, landmarks, reduction="mean")
        train_loss.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return np.mean(train_loss, dtype=np.float64)

def validate(model, loader, loss_fn, device):
    model.eval()
    val_loss = []
    for batch in tqdm.tqdm(loader, total=len(loader), desc="validation..."):
        images = batch["image"].to(device)
        landmarks = batch["landmarks"]

        with torch.no_grad():
            pred_landmarks = model(images).cpu()
        loss = loss_fn(pred_landmarks, landmarks, reduction="mean")
        val_loss.append(loss.item())

    return np.mean(val_loss, dtype=np.float64)

def predict(model, loader, device):
    model.eval()
    predictions = np.zeros((len(loader.dataset), NUM_PTS, 2))
    for i, batch in enumerate(tqdm.tqdm(loader, total=len(loader), desc="test prediction...")):
        images = batch["image"].to(device)

        with torch.no_grad():
            pred_landmarks = model(images).cpu()
            pred_landmarks = pred_landmarks.numpy().reshape((len(pred_landmarks), NUM_PTS, 2))  # B x NUM_PTS x 2

        fs = batch["scale_coef"].numpy()  # B
        margins_x = batch["crop_margin_x"].numpy()  # B
        margins_y = batch["crop_margin_y"].numpy()  # B
        prediction = restore_landmarks_batch(pred_landmarks, fs, margins_x, margins_y)  # B x NUM_PTS x 2
        predictions[i * loader.batch_size: (i + 1) * loader.batch_size] = prediction

    return predictions

def create_submission(path_to_data, test_predictions, path_to_submission_file):
    test_dir = os.path.join(path_to_data)

    output_file = path_to_submission_file
    wf = open(output_file, 'w')
    wf.write(SUBMISSION_HEADER)

    mapping_path = os.path.join(test_dir, 'test_points.csv')
    mapping = pd.read_csv(mapping_path, delimiter=',')
    
    for i, row in mapping.iterrows():
        
        
        file_name = row[0]

        point_index_list = np.array(eval(row[1]))
        points_for_image = test_predictions[i]
        needed_points = points_for_image[point_index_list].astype(np.int)
        wf.write(file_name + ',' + ','.join(map(str, needed_points.reshape(2 * len(point_index_list)))) + '\n')


In [0]:
train_transforms = transforms.Compose([
        ScaleMinSideToSize((CROP_SIZE, CROP_SIZE)),
        CropCenter(CROP_SIZE),
        TransformByKeys(transforms.ToPILImage(), ("image",)),
        TransformByKeys(transforms.ToTensor(), ("image",)),
        TransformByKeys(transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ("image",)),
    ])


test_transforms = transforms.Compose([
        ScaleMinSideToSize((CROP_SIZE, CROP_SIZE)),
        CropCenter(CROP_SIZE),
        TransformByKeys(transforms.ToPILImage(), ("image",)),
        TransformByKeys(transforms.ToTensor(), ("image",)),
        TransformByKeys(transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ("image",)),
    ])

In [16]:
train_dataset = LandmarksDataset(os.path.join(TRAIN_PATH), train_transforms, split="train")
train_dataloader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=0, pin_memory=True,
                                   shuffle=True, drop_last=True)
val_dataset = LandmarksDataset(os.path.join(TRAIN_PATH), train_transforms, split="val")
val_dataloader = data.DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=0, pin_memory=True,
                                 shuffle=False, drop_last=False)

1375it [00:00, 2945.43it/s]
2001it [00:00, 16154.05it/s]


In [17]:
gc.collect()

35

In [0]:
import torchvision.models as models

In [28]:
model = models.resnext50_32x4d(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2 * NUM_PTS, bias=True)
model.to(device)

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/checkpoints/resnext50_32x4d-7cdf4587.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [0]:
LR = 1e-3
EPOCHS = 5

In [0]:
optimizer = optim.Adam(model.parameters(), lr=LR, amsgrad=True)
loss_fn = fnn.mse_loss

In [0]:
# Tensorboard + нормальный трейнлуп с параллельной валидацией
# Нормальная модель в т.ч. обученная на лицах
# Albumenntations
# tqdm.notebook
# Выглядит, как что-то очень неэффективное(

In [34]:
print("Ready for training...")
best_val_loss = np.inf
for epoch in range(5):
    train_loss = train(model, train_dataloader, loss_fn, optimizer,  device=device)
    val_loss = validate(model, val_dataloader, loss_fn, device=device)
    
    print("Epoch #{:2}:\ttrain loss: {:10.7}\tval loss: {:10.7}".format(epoch, train_loss, val_loss))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        with open(f"model.pth", "wb") as fp:
            torch.save(model.state_dict(), fp)



training...:   0%|          | 0/24 [00:00<?, ?it/s]

Ready for training...




training...:   4%|▍         | 1/24 [00:03<01:27,  3.81s/it]

training...:   8%|▊         | 2/24 [00:07<01:21,  3.71s/it]

training...:  12%|█▎        | 3/24 [00:11<01:18,  3.74s/it]

training...:  17%|█▋        | 4/24 [00:14<01:12,  3.60s/it]

training...:  21%|██        | 5/24 [00:17<01:06,  3.49s/it]

training...:  25%|██▌       | 6/24 [00:21<01:04,  3.57s/it]

training...:  29%|██▉       | 7/24 [00:24<00:59,  3.47s/it]

training...:  33%|███▎      | 8/24 [00:28<00:57,  3.61s/it]

training...:  38%|███▊      | 9/24 [00:31<00:53,  3.55s/it]

training...:  42%|████▏     | 10/24 [00:35<00:50,  3.63s/it]

training...:  46%|████▌     | 11/24 [00:39<00:47,  3.64s/it]

training...:  50%|█████     | 12/24 [00:43<00:43,  3.63s/it]

training...:  54%|█████▍    | 13/24 [00:46<00:39,  3.56s/it]

training...:  58%|█████▊    | 14/24 [00:49<00:34,  3.48s/it]

training...:  62%|██████▎   | 15/24 [00:53<00:31,  3.49s/it]

training...:  67%|██████▋   | 16/24 [00:56<00:27,  3.47s/it]

training...:  7

Epoch # 0:	train loss:   272.8394	val loss:   280.1098




training...:   4%|▍         | 1/24 [00:03<01:26,  3.76s/it]

training...:   8%|▊         | 2/24 [00:07<01:23,  3.77s/it]

training...:  12%|█▎        | 3/24 [00:11<01:18,  3.75s/it]

training...:  17%|█▋        | 4/24 [00:15<01:16,  3.82s/it]

training...:  21%|██        | 5/24 [00:18<01:11,  3.75s/it]

training...:  25%|██▌       | 6/24 [00:23<01:09,  3.89s/it]

training...:  29%|██▉       | 7/24 [00:26<01:04,  3.78s/it]

training...:  33%|███▎      | 8/24 [00:30<01:03,  3.94s/it]

training...:  38%|███▊      | 9/24 [00:34<00:57,  3.80s/it]

training...:  42%|████▏     | 10/24 [00:37<00:51,  3.65s/it]

training...:  46%|████▌     | 11/24 [00:41<00:47,  3.62s/it]

training...:  50%|█████     | 12/24 [00:44<00:41,  3.47s/it]

training...:  54%|█████▍    | 13/24 [00:47<00:37,  3.45s/it]

training...:  58%|█████▊    | 14/24 [00:51<00:34,  3.43s/it]

training...:  62%|██████▎   | 15/24 [00:54<00:30,  3.39s/it]

training...:  67%|██████▋   | 16/24 [00:57<00:25,  3.22s/it]

training...:  7

...

Epoch # 2:	train loss:   240.5679	val loss:   234.9338




training...:   4%|▍         | 1/24 [00:04<01:32,  4.01s/it]

training...:   8%|▊         | 2/24 [00:07<01:22,  3.74s/it]

training...:  12%|█▎        | 3/24 [00:10<01:19,  3.77s/it]

training...:  17%|█▋        | 4/24 [00:14<01:13,  3.68s/it]

training...:  21%|██        | 5/24 [00:18<01:11,  3.77s/it]

training...:  25%|██▌       | 6/24 [00:21<01:06,  3.70s/it]

training...:  29%|██▉       | 7/24 [00:25<00:59,  3.51s/it]

training...:  33%|███▎      | 8/24 [00:28<00:55,  3.46s/it]

training...:  38%|███▊      | 9/24 [00:31<00:49,  3.30s/it]

training...:  42%|████▏     | 10/24 [00:34<00:46,  3.32s/it]

training...:  46%|████▌     | 11/24 [00:38<00:44,  3.46s/it]

training...:  50%|█████     | 12/24 [00:42<00:42,  3.56s/it]

training...:  54%|█████▍    | 13/24 [00:45<00:38,  3.48s/it]

training...:  58%|█████▊    | 14/24 [00:49<00:35,  3.52s/it]

training...:  62%|██████▎   | 15/24 [00:52<00:31,  3.47s/it]

training...:  67%|██████▋   | 16/24 [00:55<00:27,  3.42s/it]

training...:  7

Epoch # 3:	train loss:   212.1433	val loss:   197.3127




training...:   4%|▍         | 1/24 [00:03<01:20,  3.50s/it]

training...:   8%|▊         | 2/24 [00:06<01:13,  3.36s/it]

training...:  12%|█▎        | 3/24 [00:11<01:17,  3.70s/it]

training...:  17%|█▋        | 4/24 [00:14<01:14,  3.74s/it]

training...:  21%|██        | 5/24 [00:18<01:11,  3.76s/it]

training...:  25%|██▌       | 6/24 [00:21<01:04,  3.59s/it]

training...:  29%|██▉       | 7/24 [00:25<01:00,  3.53s/it]

training...:  33%|███▎      | 8/24 [00:28<00:56,  3.53s/it]

training...:  38%|███▊      | 9/24 [00:31<00:51,  3.41s/it]

training...:  42%|████▏     | 10/24 [00:35<00:49,  3.57s/it]

training...:  46%|████▌     | 11/24 [00:39<00:46,  3.56s/it]

training...:  50%|█████     | 12/24 [00:43<00:44,  3.70s/it]

training...:  54%|█████▍    | 13/24 [00:46<00:39,  3.59s/it]

training...:  58%|█████▊    | 14/24 [00:50<00:35,  3.59s/it]

training...:  62%|██████▎   | 15/24 [00:53<00:31,  3.53s/it]

training...:  67%|██████▋   | 16/24 [00:56<00:27,  3.40s/it]

training...:  7

Epoch # 4:	train loss:   192.4275	val loss:   269.1752


In [35]:
test_dataset = LandmarksDataset(os.path.join(TEST_PATH), test_transforms, split="test")
test_dataloader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=0, pin_memory=True,
                                  shuffle=False, drop_last=False)

with open(f"model.pth", "rb") as fp:
    best_state_dict = torch.load(fp, map_location="cpu")
    model.load_state_dict(best_state_dict)

test_predictions = predict(model, test_dataloader, device)



331it [00:00, 80594.14it/s]


test prediction...:   0%|          | 0/6 [00:00<?, ?it/s]

test prediction...:  17%|█▋        | 1/6 [00:24<02:03, 24.61s/it]

test prediction...:  33%|███▎      | 2/6 [00:42<01:30, 22.63s/it]

test prediction...:  50%|█████     | 3/6 [01:02<01:05, 21.67s/it]

test prediction...:  67%|██████▋   | 4/6 [01:21<00:41, 20.95s/it]

test prediction...:  83%|████████▎ | 5/6 [01:40<00:20, 20.27s/it]

test prediction...: 100%|██████████| 6/6 [01:42<00:00, 17.12s/it]


In [0]:
with open(global_path+"test_predictions.pkl", "wb") as fp:
    pickle.dump({"image_names": test_dataset.image_names,
                 "landmarks": test_predictions}, fp)

create_submission(TEST_PATH, test_predictions, f"submit.csv")